# import các thư viện

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

from urllib import parse
from urllib import robotparser

# Phần 1: Thu Thập Dữ Liệu
 - Nhóm thực hiện việc thu thập dữ liệu từ 3 trang web bán điện thoại tại Việt Nam, bao gồm: https://didongthongminh.vn/dien-thoai,  https://www.nguyenkim.com/dien-thoai-di-dong,  https://www.thegioididong.com/dtdd .
 Viêc thu thập dẽ liệu từ 3 trang web này được check robotparser như sau: 

### Check robotparser

In [2]:
urls = ['https://www.nguyenkim.com/dien-thoai-di-dong',
       'https://www.nguyenkim.com/dien-thoai-di-dong',
       'https://www.thegioididong.com/dtdd']
for url in urls:
    URL_BASE = url
    parser = robotparser.RobotFileParser()
    parser.set_url(parse.urljoin(URL_BASE, 'robots.txt'))
    parser.read()
    print(url,'check = ', parser.can_fetch('*', url))

https://www.nguyenkim.com/dien-thoai-di-dong check =  True
https://www.nguyenkim.com/dien-thoai-di-dong check =  True
https://www.thegioididong.com/dtdd check =  True


Kết quả check ở 3 trang web là True nên ta có thể thu thập dữ liệu hợp pháp từ 3 trang này.

###  Thu thập dữ liệu từ trang web https://didongthongminh.vn/dien-thoai
 - Thực hiện bằng phương pháp parser html

In [3]:
# Đây là các thuộc tính cơ bản của một điện thoại.
# Nhóm sẽ tiến hành thu thập các thông tin bao gồm: tên, ram, rom, dung lượng pin, màn hình, cpu, camera chính
# và camera selfie và giá thành(đây là thuộc tính dự đoán)

Name = []
Ram = []
Rom =[]
Pin = []
Screen = []
Cpu = []
Main_Camera = []
Selfie_Camera = []
Price =[]
i = 1
while True:
    url = f'https://didongthongminh.vn/dien-thoai/page/{i}'
    i +=1
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text, "html.parser")
    phone_list = tree.find_all('div',{'class':'dst_hproduct_hidden'})
    if len(phone_list) == 0:
        break
    for phone in phone_list:
        # Tên điện thoại.
        try:
            name = phone.find('a',{'class':'dst_hproduct_title'}).text
        except:
            name = ''
        Name.append(name)
        configuration = phone.find_all('span',{'class':'dst_cauhinh'})
        # Màn hình 
        try:
            screen = configuration[0].text[configuration[0].text.find(":")+1:]
        except:
            screen = ''
        Screen.append(screen)
        # CPU
        try:
            cpu = configuration[1].text[configuration[1].text.find("CPU:")+len("CPU: "):]
        except:
            cpu = ''
        Cpu.append(cpu)
        # Camera
        try:
            main_c = configuration[3].text[configuration[3].text.find("Camera:")+len("Camera:")+1:configuration[3].text.find("Selfie:")-2]
            selfie_c = configuration[3].text[configuration[3].text.find("Selfie:")+len("Selfie:")+1:]
        except:
            main_c =''
            selfie_c =''
        Main_Camera.append(main_c)
        Selfie_Camera.append(selfie_c)
        # Ram
        try:
            ram = configuration[2].text[len('Ram:'):configuration[2].text.find(',')]
        except:
            ram = ''
        Ram.append(ram)
        # Rom
        try:
            rom = configuration[2].text[configuration[2].text.rfind(':')+1:]
        except:
            rom = ''
        Rom.append(rom)
        # dung lượng Pin
        try:
            pin = configuration[4].text[len("Pin: "):]
        except:
            pin = ''
        Pin.append(pin)
        # giá điện thoại.
        try:
            price = phone.find('div',{'class':"dst_hproduct_price"}).text
        except:
            price = ''
        Price.append(price)

### Thu thập dữ liệu từ trang web: https://www.nguyenkim.com/dien-thoai-di-dong

In [4]:
i = 1
while True:
    url = f'https://www.nguyenkim.com/dien-thoai-di-dong/page-{i}/'
    i +=1
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text, "html.parser")
    urls = tree.find_all('a', {'class': 'nk-link-product'})
    if len(urls) == 0:
        break
    for u in urls:
        phone_html_text = requests.get(u['href']).text
        phone_tree = BeautifulSoup(phone_html_text, "html.parser")
        
        # Tên điện thoại
        try:
            name = phone_tree.find('h1',{'class':'product_info_name'}).text.replace("Điện thoại ","")
        except:
            name = ''
        Name.append(name)
        
        configurations = phone_tree.find('tbody',{'class':'popup'}).find_all('tr') # thông số kỹ thuật.
        conf = []
        for _conf in configurations:
            s = _conf.text.split(":")
            conf.append((s[0],s[1]))
        conf = dict(conf)
        
        # Màn hình
        try:
            screen = conf["Kích thước màn hình"]
        except:
            screen =''
            
        Screen.append(screen)
            
        # Cpu
        try:
            cpu = conf["Chipset"]
        except:
            cpu = ''
        Cpu.append(cpu)
        
        # Camera
        try:
            selfie_c = conf["Camera trước"]
        except:
            selfie_c =''
        Selfie_Camera.append(selfie_c)
                             
        try:
            main_c = conf["Camera sau"]
        except:
            main_c =''
        Main_Camera.append(main_c)
        
        # Ram
        try:
            ram = conf["RAM"]
        except:
            ram =''
        Ram.append(ram)
        
        #Rom
        try:
            rom = conf["Bộ nhớ trong"]
        except:
            rom = ''
        Rom.append(ram)
        
        # dung lượng Pin
        try:
            pin = conf["Dung lượng Pin"]
        except:
            pin =''
        Pin.append(pin)
        
        # giá điện thoại
        try:
            price = phone_tree.find('div',{'class':'product_info_price_value-final'}).text
        except:
            price = ''
        Price.append(price)


### Thu thập dữ liệu từ web: https://www.thegioididong.com/dtdd
- Việc tải nhiều trang cho các điện thoại khác nhau nên phải tương tác với trang web, do đó nhóm đã sử dụng phương pháp webdriver để tải các trang và thu thập dữ liệu.

In [5]:
# Mở Chrome và cho full màn hình
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
browser.maximize_window()
# Mở trang chủ của Vietjet
browser.get('https://www.thegioididong.com/dtdd')
while True:
    time.sleep(1)
    try:
        browser.find_element_by_css_selector('a.viewmore').click()
    except:
        break
        
html_text = browser.page_source
browser.quit()
tree = BeautifulSoup(html_text, 'html.parser')
phones = tree.find_all('li',{'class':'item'})
for phone in phones:
    url = phone.find('a')['href']
    url='https://www.thegioididong.com'+url
    phone_html_text = requests.get(url).text
    tree = BeautifulSoup(phone_html_text, 'html.parser')
    #name
    try:
        name=tree.find('div',{'class':'rowtop'}).h1.text.replace('Điện thoại ','')
    except:
        name=''
    parameter=tree.find('ul',{'class':'parameter'})
    #màn
    try:
        screen=parameter.find('li',{'class':'g6459_79_77'}).div.text
    except:
        screen=''
    #cpu
    try:
        cpu=parameter.find('li',{'class':'g6059'}).div.text
    except:
        cpu=''
    #caera trước
    try:
        main_camera=parameter.find('li',{'class':'g27'}).div.text
    except:
        main_camera=''
    #camera sau
    try:
        selfie_camera=parameter.find('li',{'class':'g29'}).div.text
    except:
        selfie_camera=''
    #rom
    try:
        rom=parameter.find('li',{'class':'g49'}).div.text
    except:
        rom=''
    #ram
    try:
        ram=parameter.find('li',{'class':'g50'}).div.text
    except:
        ram=''
    #pin
    try:
        pin=parameter.find('li',{'class':'g84_10882'}).div.text
    except:
        pin=''
    #giá
    try:
        price=tree.find('div',{'class':'area_price'}).strong.text
    except:
        price=''
    Name.append(name)
    Screen.append(screen)
    Cpu.append(cpu)
    Main_Camera.append(main_camera)
    Selfie_Camera.append(selfie_camera)
    Rom.append(rom)
    Ram.append(ram)
    Pin.append(pin)
    Price.append(price)

In [6]:
df = pd.DataFrame({'Name' : pd.Series(Name),
                   'Screen' : pd.Series(Screen),
                   'Cpu' : pd.Series(Cpu),
                   'MainCamera' : pd.Series(Main_Camera),
                   'SelfieCamera' : pd.Series(Selfie_Camera),
                   'Rom' : pd.Series(Rom),
                   'Ram' : pd.Series(Ram),
                   'Pin' : pd.Series(Pin),
                   'Price': pd.Series(Price)})

In [7]:
df

,Name,Screen,Cpu,MainCamera,SelfieCamera,Rom,Ram,Pin,Price
0,Xiaomi POCO M3 - Chính hãng,"IPS LCD, 6.53"", Full HD+",Snapdragon 662 8 nhân,"Chính 48 MP & Phụ 2 MP, 2 MP",8MP,128/64 GB,4GB,6000mAh,3.490.000 Đ
1,Xiaomi Mi 11 128Gb Ram 8Gb,"AMOLED, 6.81"", Quad HD+ (2K+)",Snapdragon 888 (5 nm),"Chính 108 MP & Phụ 13 MP, 5 MP",20 MP,128 GB,8 GB,Li-Ion 4600 mAh,16.490.000 Đ
2,Oppo Reno5 8GB/128GB chính hãng,"6,43 inch, OLED",Qualcomm SM7125 Snapdragon 720G (8 nm),"Chính 64 MP & Phụ 8 MP, 2 MP, 2 MP",44MP,128 GB,8 GB,4.310 mAh + Sạc nhanh 50W,8.690.000 Đ
3,Redmi Note 9 Pro 5G 6Gb/128Gb,,Octa-core (2x2.2 GHz Kryo 570 & 6x1.8 GHz Kryo...,Chính 108MP+8MP+2MP+2MP,16MP,128 GB,6GB,Li-Po 4820 mAh,6.390.000 Đ
4,iPhone SE Cũ 16Gb Nguyên Bản,"IPS LCD, 4.0"", DVGA",Apple A9,12 MP,1.2 MP,16 GB,2 GB,Li-Po 1624 mAh,2.190.000 Đ
...,...,...,...,...,...,...,...,...,...
1837,Itel it2171,,,0.3 MP,,,,,210.000₫
1838,Mobell C310,,,0.8 MP,,,,,200.000₫
1839,Masstel IZI 112,,,,,,,,200.000₫
1840,Masstel IZI 120,,,,,,,,190.000₫


- Loại bỏ các dòng lặp.

In [8]:
df = df.drop_duplicates()

In [9]:
df

,Name,Screen,Cpu,MainCamera,SelfieCamera,Rom,Ram,Pin,Price
0,Xiaomi POCO M3 - Chính hãng,"IPS LCD, 6.53"", Full HD+",Snapdragon 662 8 nhân,"Chính 48 MP & Phụ 2 MP, 2 MP",8MP,128/64 GB,4GB,6000mAh,3.490.000 Đ
1,Xiaomi Mi 11 128Gb Ram 8Gb,"AMOLED, 6.81"", Quad HD+ (2K+)",Snapdragon 888 (5 nm),"Chính 108 MP & Phụ 13 MP, 5 MP",20 MP,128 GB,8 GB,Li-Ion 4600 mAh,16.490.000 Đ
2,Oppo Reno5 8GB/128GB chính hãng,"6,43 inch, OLED",Qualcomm SM7125 Snapdragon 720G (8 nm),"Chính 64 MP & Phụ 8 MP, 2 MP, 2 MP",44MP,128 GB,8 GB,4.310 mAh + Sạc nhanh 50W,8.690.000 Đ
3,Redmi Note 9 Pro 5G 6Gb/128Gb,,Octa-core (2x2.2 GHz Kryo 570 & 6x1.8 GHz Kryo...,Chính 108MP+8MP+2MP+2MP,16MP,128 GB,6GB,Li-Po 4820 mAh,6.390.000 Đ
4,iPhone SE Cũ 16Gb Nguyên Bản,"IPS LCD, 4.0"", DVGA",Apple A9,12 MP,1.2 MP,16 GB,2 GB,Li-Po 1624 mAh,2.190.000 Đ
...,...,...,...,...,...,...,...,...,...
1837,Itel it2171,,,0.3 MP,,,,,210.000₫
1838,Mobell C310,,,0.8 MP,,,,,200.000₫
1839,Masstel IZI 112,,,,,,,,200.000₫
1840,Masstel IZI 120,,,,,,,,190.000₫


- Ghi thông tin vào file data01.csv để dễ xử lý về sau.

In [10]:
export_csv = df.to_csv ('data01.csv', index = None, header=True)